In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
## Analyze the selected song data and prepare it for the model 
## This notebook is used to prepare the data for the model. It is not used in the model itself. 
## The data is prepared in the following steps:
## 1. Load the data

## load the midi files from the songs folder 

import os 
import glob 
import pickle 
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import music21 as m21

from tqdm import tqdm
from music21 import converter, instrument, note, chord, stream
from keras.utils import np_utils
import json
import IPython.display
from datetime import datetime
import random

import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from tqdm.notebook import tqdm, trange

import random
import itertools

In [ ]:

!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install music21
!pip install pypianoroll

## Getting the Lakh MIDI Dataset ##

source: https://nbviewer.org/github/craffel/midi-dataset/blob/master/Tutorial.ipynb 

In [6]:
import os 
root_dir = '..\TaylorSwiftBot' # change this to your root directory 
# get the lmd_full.tar.gz from metadata folder 
# get the lmd_matched.tar.gz from metadata folder
data_dir = os.path.join(root_dir, 'data') # change this to your data directory 
music_dataset_lpd_dir = os.path.join(data_dir, 'lmd_matched') # change this to your lmd_matched directory 

RESULTS_PATH = os.path.join(root_dir, 'Lakh Piano Dataset', 'Metadata')

# Utility functions for retrieving paths
def msd_id_to_dirs(msd_id):
    """Given an MSD ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return os.path.join(msd_id[2], msd_id[3], msd_id[4], msd_id)


def msd_id_to_h5(msd_id):
    """Given an MSD ID, return the path to the corresponding h5"""
    return os.path.join(RESULTS_PATH, 'lmd_matched_h5',
                        msd_id_to_dirs(msd_id) + '.h5')

# Load the midi npz file from the LMD cleansed folder
def get_midi_npz_path(msd_id, midi_md5):
    return os.path.join(data_dir,
                        msd_id_to_dirs(msd_id), midi_md5 + '.npz')
    
# Load the midi file from the Music Dataset folder
def get_midi_path(msd_id, midi_md5):
    return os.path.join(music_dataset_lpd_dir,
                        msd_id_to_dirs(msd_id), midi_md5 + '.mid')

ModuleNotFoundError: No module named 'request'